# <center>Big Data for Engineers &ndash; Exercises &ndash; Solution</center>
## <center>Spring 2025 &ndash; Week 5 &ndash; ETH Zurich</center>
## <center>Wide Column Stores - HBase</center>

In this exercise sheet you will analyse some important architectural components of HBase. The following is a brief summary of the topics:
* Region Servers
* HFile Indexes
* Bloom Filters
* Log-Structured Merge Trees
* Write Ahead Logs

## Exercise 1 &mdash; Inside a RegionServer

In this exercise you will see how a RegionServer in HBase would execute a query.

Imagine that we have an HBase table called '`phrases`', which has the following schema:

* Column family: `words`
  * column: A
  * column: B
  * column: C
  * (potentially also columns D, E, F, etc.)

Thus, the table has only one column family. Each column in this family holds one word.

Recall from the lecture slides that keys in HBase have the following structure:

<img src="https://polybox.ethz.ch/index.php/s/XjO0LM1r6L10kT3/download" width="70%">

We will make some simplifications to keys format to avoid excessive clutter in this exercise. Since the table in this exercise has only one column family, we will omit it from the key and only specify the column name (A,B,C, ...). We will also omit the length fields and the "key type" field. The timestamp field in this exercise will contain integers from 1 to 10 (in reality, the timestamp would contain the number of milliseconds). Thus, keys that will be used in this exercise consist of three fileds: **row, column, timestamp**.

### Tasks to do

State which Key-Value pairs will be returned by each of the following queries. Assume that the HBase instance is configured to return only the latest version of a cell.

1. `get 'phrases', '278'`
1. `get 'phrases', '636'`
1. `get 'phrases', '593'`
1. `get 'phrases', '640'`
1. `get 'phrases', '443'`

To answer this question, use the diagram below, which represents the state of a RegionServer responsible for the row region in the range of row IDs 100&ndash;999, which is the region into which all these queries happen to fall. 

<img src="https://polybox.ethz.ch/index.php/s/mbc7boFDDCo2V9D/download" >

You can format your answer for this exercise as follows (doulbe-click to edit)

1. get 'phrases', 'row_id' 

| Row | Column | Timestamp | Value | Where it came from (which HFile) |
|:-----:|:-----:|:-----:|:-------|:--------------------:|
||||||
||||||
||||||

## Solution to Exercise 1

1. get 'phrases', '278' 

| Row | Column | Timestamp | Value | Where it came from |
|:-----:|:-----:|:-----:|:-------|:--------------------:|
|278|A|8|cake|HFile1|
|278|B|1|is|HFile2|
|278|C|5|a lie|HFile3|

2. get 'phrases, '636'

| Row | Column | Timestamp | Value | Where it came from |
|:-----:|:-----:|:-----:|:-------|:--------------------:|
|636|A|4|watch|HFile2|
|636|B|1|your|MemStore|
|636|C|1|step|MemStore|

3. get 'phrases', '593'

| Row | Column | Timestamp | Value | Where it came from |
|:-----:|:-----:|:-----:|:-------|:--------------------:|
|593|A|1|indeed|HFile2|

4. get 'phrases', '640'

| Row | Column | Timestamp | Value | Where it came from |
|:-----:|:-----:|:-----:|:-------|:--------------------:|
|640|A|5|long|HFile1|
|640|B|6|live|HFile2|
|640|C|2|rock'n'roll|MemStore|

Note that "640,C,1 -> the king" (HFile1) got superseded by "640,C,2 -> rock'n'roll" (Memstore) because of the more recent timestamp.

5. get 'phrases', '443'

The row doesn't exist, so no output.

## Exercise 2 &mdash; Building an HFile index

When servicing a Read request, the RegionServer needs to check its MemStore and all HFiles for the existence of the requested key. In order to avoid scanning HFiles entirely, HBase uses sparse index structures to quickly skip to the position of the *HBase block* which may hold the requested key. Note HBase block is not to be confused with HDFS block and the underlying file system block, see [here](https://blog.cloudera.com/hbase-blockcache-101/#3) for a good discussion. HBase blocks come in 4 varieties: DATA, META, INDEX, and BLOOM.

By default, each *HBase block* is 64KB (but it is configurable) in size and always contains whole key-value pairs, so, if a block needs more than 64KB to avoid splitting a key-value pair, it will just grow. 

In this task, you will be building the index of an HFile. __For the purpose of this exercise__, assume that each HBase block is 40 bytes long, and each character in keys and values is worth 1 byte: for example, the first key-value pair in the diagram below is worth $3 + 1 + 1 + 6 = 11$ bytes.

<img src="https://polybox.ethz.ch/index.php/s/Sj5PNFWcy8TbVOh/download" width="50%">

Based on the contents of the HFile above, you need to populate the index. First, calculate how the blocks will be "assembled", and then use the appropriate keys for the index.

You can use the following table (again, you can edit it by double-clicking). Use as many or as few rows as you need.

| RowId | Column | Version |
|-------|--------|---------|
|       |        |         |
|       |        |         |
|       |        |         |
|       |        |         |
|       |        |         |
|       |        |         |
|       |        |         |
|       |        |         |
|       |        |         |
|       |        |         |

## Solution to Exercise 2

<img src="https://polybox.ethz.ch/index.php/s/nUDW3hkHUbxssXL/download" width="80%">

Note that every block is at least 40 bytes long, in order to avoid splitting key-value pairs. We will end up with five blocks in total, and the index will point to the first key of every block. it will end up looking like this: 

| RowId | Column | Version |
|-------|--------|---------|
| 113   |   C    |    5    |
| 383   |   C    |    5    |
| 469   |   B    |    4    |
| 641   |   A    |    2    |
| 849   |   B    |    5    |





## Exercise 3 &mdash; Bloom filters

In **Exercise 2** we built an index on each HFile containing our data, and stored it in a separate index file. Each index file is sorted on the RowId and as we have seen, this index can map each RowId to its offset in the HFile. Now, if we want to search an ID in this index, the best we can do is a Binary Search. **Can we do better than that?**

We might wonder if there is a more efficient way to know which file might contain our required data. We don’t want to read each file, as that would be slow, and we have to read a lot of data from the disk and lead to severe **read amplification**.

We now introduce **Bloom filters**, a space-efficient probabilistic data structure used by HBase used to directly discard all get requests which query data not stored in the HFile. Bloom Filters are stored in the metadata of each HFile.

Let's start with summarizing what we have in memory when working with HBase: MemStore, LRU (Least Recently Used) BlockCache, Indices of HFiles, and Bloom filters. We did not have time to cover Bloom filters in the lecture, but they are actually very crucial in avoiding disk reads if we can guarantee that a key is **not** in an HFile. Bloom filters allow us to very quickly determine whether an element belongs to a set.

Bloom filters are a data structure used to speed up queries, useful in the case in which it's likely that the value we are looking doesn't exist in the collection we are querying. Their main component is a bit array with all values initially set to 0. When a new element is inserted in the collection, its value is first run through a certain number of (fixed) hash functions, and the locations in the bit array corresponding to the outputs of these functions are set to 1.

This means that when we query for a certain value, if the value has previously been inserted in the collection then all the locations corresponding to the hash function outputs will certainly already have been set to 1. On the contrary, if the element hasn't been previously inserted, then the locations may or may not have already been set to 1 by other elements.

Then, if prior to accessing the collection we run our queried value through the hash functions, check the locations corresponding to the outputs, and find any of them to be 0, we are guaranteed that the element is not present in the collection (No False Negatives), and we don't have to waste time looking. If the corresponding locations are all set to 1, the element may or may not be present in the collection (possibility of False Positives), but in the worst case we're just wasting time.

Inspect the following examples. Say we have hash functions that map the input `John Smith` and `Mary Smith` to the bit array `011001110000`. When we have a new input `Albert Einstein` which is mapped by the same hash functions to the bit array `000101100000`. This clearly does not correspond to the bit array produced by the previous two inputs. Hence, we can say that `Albert Einstein` is not in the set which `John Smith` and `Mary Smith` belong to (denoted as `{the Smiths}` for short). However, another input `Louis de Broglie` whose bit array after hashing is `010000110000` is then a false positive for the set `{the Smiths}`.

<img src="https://polybox.ethz.ch/index.php/s/iMEEGRSaVkRHjDG/download" width="50%">
<img src="https://polybox.ethz.ch/index.php/s/GJQgLU8W258YsJl/download" width="50%">
<img src="https://polybox.ethz.ch/index.php/s/MAQM88My5yWdTps/download" width="50%">
<img src="https://polybox.ethz.ch/index.php/s/xWrmQIPWV5iGj64/download" width="50%">

As you have seen in the task above, HBase has to check all HFiles, along with the MemStore, when looking for a particular key. As an optimisation, Bloom filters are used to avoid checking an HFile if possible. Before looking inside a particular HFile, HBase first checks the requested key against the Bloom filter associated with that HFile. If it says that the key does not exist, the file is not read.

In this task we will look at how Bloom filters work. We will use a list of words to populate a Bloom filter and we will then query it.

Bloom filter requires several hash functions. To keep things easily computable by a human, we will define the following three hash functions for the purpose of this exercise:

1. Given an English word $x$, the value of the first hash function, $hash_1(x)$, is equal to the *first letter of the word*. E.g.: $hash_1($"`federal`"$)$ = "`f`"

1. Given an English word $x$, the value of the second hash function, $hash_2(x)$, is equal to the *second letter of the word*. E.g.: $hash_2($"`federal`"$)$ = "`e`"

1. Given an English word $x$, the value of the third hash function, $hash_3(x)$, is equal to the *third letter of the word*. E.g.: $hash_3($"`federal`"$)$ = "`d`"

A Bloom filter starts with a bit array which has value "0" recorded for each possible output value of all three hash functions (or, for example, modulo the size of the bit array, if the output range of the hash functions is too large).
When we *add* an element to a Bloom filter, we compute the three values of the three hash functions and set those locations in the Bloom filter to "1". For example, if we add "`federal`" to the Bloom filter using the three hash functions that we have defined above, we get the following:

| | | |1|1|1| | | | | | | | | | | | | | | | | | | | |
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|

Here, only values "1" are displayed to avoid cluttering the view; thus, if a cell is empty, it is assumed to hold a "0".

**Here is what we will do next.**<br>
Step 1. Create a bloom filter that for a collection of words (Collection A). 
<br>
Step 2. Given this bloom filter, check if a set of new words (Collection B) are members of Collection A. 

**Step 1.** First, populate the following table with the outputs of these hash functions (double-click the table to edit it and hit Ctrl+Enter to exit the editing mode; you are also free to do this task in some other tool, of course):
[**Collection A**]

| Word    | $hash_1$ | $hash_2$ | $hash_3$ |
|:--------|-------------|-------------|-------------|
|round    |             |             |             |
|sword    |             |             |             |
|past     |             |             |             |
|pale     |             |             |             |
|nothing  |             |             |             |
|darkness |             |             |             |
|water    |             |             |             |
|feet     |             |             |             |
|thin     |             |             |             |
|passage  |             |             |             |
|corner   |             |             |             |

Now, *add* each word from the list into the following Bloom filter (you can also double-click to edit it; you can double-click the Bloom filter populated with "federal" above to see an example of a filled-in filter): [**Bloom Filter for Collection A**]

| | | | | | | | | | | | | | | | | | | | | | | | | | |
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|

2. For each word from the following list, state whether this Bloom filter reports it as belonging to the set or not (skip filling-in the hash columns, if you want): [**Collection B**]

| Word    | $hash_1$ | $hash_2$ | $hash_3$ | The Bloom filter says the word belongs to the set: (yes/no) |
|:--------|----------|----------|----------|:-----------:|
|sword    |          |          |          |             |
|sound    |          |          |          |             |
|psychic  |          |          |          |             |
|pale     |          |          |          |             |
|book     |          |          |          |             |
|deaf     |          |          |          |             |
|truss    |          |          |          |             |

Which of the words that were flagged by the Bloom filter as belonging to the set are actually **not** in the set (a *false positive* outcome)?

Which of the words that were flagged by the Bloom filter as **not** belonging to the set actually **do belong** to the set (a *false negative* outcome)?

## Solution to the Exercise 3

Hashes for each of the words from the first list: [**Collection A**]

| Word    | hash_1 | hash_2 | hash_3 |
|:--------|-------------|-------------|-------------|
|round    |R            |O            |U            |
|sword    |S            |W            |O            |
|past     |P            |A            |S            |
|pale     |P            |A            |L            |
|nothing  |N            |O            |T            |
|darkness |D            |A            |R            |
|water    |W            |A            |T            |
|feet     |F            |E            |E            |
|thin     |T            |H            |I            |
|passage  |P            |A            |S            |
|corner   |C            |O            |R            |

The Bloom filter with all those words added to it: [**Bloom Filter for Collection A**]


|1| |1|1|1|1| |1|1| | |1| |1|1|1| |1|1|1|1| |1| | | |
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|

For each word from the following list, state whether this Bloom filter marks them as belonging to the set or not:
[**Collection B**]

| Word    | The Bloom filter says the word belongs to the set: (yes/no) | Comment |
|:--------|:-----------:|:-------------|
|sword    | yes            | present in the original list |
|sound    | yes            | not from the original list, but still got flagged as present: this is a *false positive* |
|psychic  | no             | indeed, it's not from the original list |
|pale     | yes            | present in the original list |
|book     | no             | indeed, it's not from the original list |
|deaf     | yes            | a *false positive* |
|truss    | yes            | a *false positive* |

As you can see, a Bloom filter can produce false positive outcomes. Luckily, it never produces false negative outcomes, i.e., if a Bloom filter says that an element is absent from the set, it is guaranteed to really be absent from the set.

## Exercise 4 &mdash; Log-structured merge-tree (LSM tree)

So far, the optimisations we have done above can be summarised as follows:

1. **Sparse Index** (HFile index)  
   A small, in-memory index that stores pointers. This helps quickly locate and jumpstart the approximate position of a key in the HFile before doing a more precise binary search.

2. **Bloom Filter**  
   A **probabilistic** data structure that can definitively say if a key **does not** exist in a given HFile (no false negatives). If the Bloom filter says "not present," the HFile can be skipped entirely. If it says "might be present," a normal lookup is done.


In sections 1 to 3 we have essentially seen methods to optimise reads on our HFiles. **But how are the HFiles created in the first place?**
Essentially, the whole HBase system composed of MemStores, HFiles, HFile indexes is an implementation of a very common pattern called [**log-structured merge-trees**](https://en.wikipedia.org/wiki/Log-structured_merge-tree). 

A **Log-Structured Merge (LSM) Tree** is a type of database index optimized for write-intensive workloads. It relies on maintaining an in-memory data structure (often a balanced binary search tree or skip list) for fast writes and periodically flushing that data to disk in sorted files.

Before we explain how LSM Trees function, we give a mapping from common terminology to HBase specific naming:

**Common -> HBase:**<br>
MemTable -> MemStore<br>
SSTable -> HFile<br>
WAL -> HLog<br>


#### 1. **In-Memory Component (MemTable)**

At the heart of an LSM Tree is an **in-memory balanced search tree** (e.g., a Red-Black Tree, AVL Tree, or sometimes a skip list). This structure, commonly called a **MemTable**, provides:

- **Fast writes:** O(log n) insertion/update in memory.
- **Fast reads on recent data:** O(log n) lookups for keys currently in memory.

However, this approach has two main issues:

1. **Lack of Durability**  
   If the system crashes, everything in memory is lost.

2. **Limited Memory and Excessive Growth**  
   The in-memory tree can grow very large and must eventually be offloaded to disk to free up RAM.

Due to the above reasons, we have to periodically flush the data on disk.

#### 2. **Flushing to SSTables**

When the in-memory tree **exceeds a certain size** threshold, the database **flushes** it to disk, creating an **SSTable (Sorted String Table)**.

1. Perform an **in-order traversal** of the balanced tree to produce a sorted list in **O(n)** time.  
2. Write this sorted list to a **new SSTable file** on disk.  
3. **Clear** the in-memory tree (memtable) to accept new writes.

**SSTables are immutable** after creation. Future changes to the same keys appear in newer SSTables.

Over time, many SSTables may accumulate on disk. **Reads** must check:

1. **In-Memory Tree**: O(log n) lookup. In case of a miss, we then proceed to check On-Disk.
2. **On-Disk SSTables**: Search from **newest** to **oldest**. Each SSTable is sorted, so it can be searched with **binary search** (O(log n) per SSTable).

If a key is found in a recent SSTable, there is no need to check older SSTables. Because SSTables are immutable, a newer table always has the most up-to-date version of a key.




#### 3. **Deletions via Tombstones**

As we have mentioned, our SSTables are immutable once written to disk. To handle deletions, LSM Trees do not immediately remove old records. Instead, a **tombstone** (a special marker) is written for a key that is deleted. The tombstone indicates that any earlier entries for that key should be ignored.

- **Old SSTables** still contain the key.  
- **The newer SSTable** has the tombstone, effectively overriding the older version.

A tombstone is similar to the idea of a **“soft delete”** from the relational database world. When we delete data, HBase does not delete it right away, instead associates a tombstone with it. In other words, a tombstone is a marker that is kept to indicate data that has been deleted. When we execute a delete operation it’s instead treated as an update operation that places a
tombstone on the value. Tombstones are removed as part of major compaction: during a major compaction, any row with an expired tombstone will not be propagated further.

Tombstones are a solution for deletes on LSM Trees, but they cause the following problems:

1. As a tombstone itself is a record, it takes storage space. Hence, it should be kept in mind that **upon deletion, the application will end up increasing the data size** instead of shrinking it. Furthermore, if there are a lot of tombstones, the available storage for the application could be substantially reduced.
2. When a table accumulates many tombstones, read queries on that table could become slow and can cause serious performance problems like timeouts. This is because we have to read much more data until an actual major compaction happens and removes the tombstones (and major compactions happen very infrequently, in the order of only once a week).


#### 4. **Compaction**

Since SSTables are immutable, over time:

- **Tombstoned keys** accumulate.  
- **Multiple versions** of the same key may exist in different SSTables.

**Compaction** merges these SSTables, with the following procedure:

1. **Merge Sort** approach: Because each SSTable is sorted, merging them is **O(n)** overall (where n is the combined size of the SSTables).  
2. **Discard old versions and tombstones** during the merge, creating a new, consolidated SSTable.  
3. Remove the original SSTables.

Compaction **reduces storage overhead** and **improves read performance** by minimizing the number of files to check.

In HBase, there are two types of compaction. **Minor Compactions** usually merge a small number of HFiles in a Region, and they do not remove tombstones. **Major Compactions** merge **all** HFiles in a Region, and therefore can safely remove tombstones. As the operation is extremely expensive, it is usually ran infrequently (by default every seven days).

#### 5. **Exercise**

In summary, an LSM tree is highly efficient in applications using wide column storage such as HBase, Cassandra, BigTable, LevelDB, where insertions in memory happen quite often.

The following figure is from the HBase Guide book where we see how a multipage block is merged from the in-memory tree into the next on-disk tree. Trees in the store files are arranged similar to B-trees. Merging writes out a new block with the combined result. Eventually, the trees are merged into the larger blocks.<br>

<img src="https://polybox.ethz.ch/index.php/s/4o2fzmVJVbV7U1j/download" width="60%">


Inserting data into LSM Tree:

1. When a write comes, it is inserted in the memory-resident MemStore.
2. When the size of the MemStore exceeds a certain threshold, it’s flushed to the disk.
3. As MemStore is already sorted, creating a new HFile segment from it is efficient enough.
4. Old HFiles are periodically compacted together to save disk space and reduce fragmentation of data.

Reading data from LSM Tree:

1. A given key is first looked up in the MemStore.
1. Then using a hash index it’s searched in one or more HFiles depending upon the status of the compaction.

We will now walk through a concrete exercise to understand how LSM tree works in HBase. Image we have a client who is constantly writing into HBase. The client also occasionally reads and deletes key value pairs in HBase. MemStore and disk are two storages we examine in this assignment. 

<img align="center" width="50%" src="https://polybox.ethz.ch/index.php/s/l1JTs41GhLE1RlK/download"><br>

The client requests the following operations from HBase in sequence:
1. writing the following key value pairs into HBase: `(C,1), (B,2), (A,9), (A,109), (G,8), (D,67), (Z,0)`;
2. reading the value of key `A`;
3. deleting the key `Z`;
4. writing the following key value pairs: `(S,100), (Z1,900), (A1,9), (A01,1), (Z1,850)`. 

In the meanwhile, in HBase flush and compaction are conducted to optimize transfer. 

Please draw the processes of 
1. how the key value pairs are stored in HBase? To simplify the actual key in HBase, we use key[t] to denote the key. E.g., when the client writes the key value pair `(C,1)` into HBase, it is first stored in MemStore with the key value `C[t1] 1`.

<img align="center" width="50%" src="https://polybox.ethz.ch/index.php/s/tHMLCXeJZ2IW8Yp/download"><br>

2. how does HBase flush and compact the HFiles? Let us set the threshold of flush to three, i.e., when the key value pairs in MemStore have reached three, HBase will flush them to disk (from $C_0$ to $C_1$). Let us also set the threshold of compaction to three, i.e., if there exist two HFiles, each with three key value pairs, we have to compact them into a bigger HFile (from $C_1$ to $C_2$). The same rule applies for bigger HFiles: whenever there is of factor two some HFile at the level of $C_{k-1}$, compact them to the level $C_k$. 

## Solution to Exercise 4
Here is a possible chain of operations that would happen. Please read the charts from left to right in a row and then go to the next row.

<img align="left" width="50%" src="https://polybox.ethz.ch/index.php/s/zlwm8lw7yBLfMZB/download">

## Exercise 5 &mdash; Write-Ahead Log (WAL)

Multiple times in this course we have mentioned that machines can fail or restart anytime. If a program is in the middle of performing a data modification, what will happen when the machine it is running on loses power? When the machine restarts, the program might need to know the last thing it was doing. Based on its atomicity and durability needs, the program might need to decide to redo or undo or finish
what it had started. How can the program know what it was doing before the system crash?

To guarantee durability and data integrity, each modification to the system is first written to an **append-only log on the disk**. This log is known as **WriteAhead Log (WAL)** or **transaction log** or **commit log** or **HLog** in the case of HBase. Writing to the WAL guarantees that if the machine crashes, the system will be able to recover and reapply the operation if necessary.

The key idea behind the WAL is that all modifications before they are applied to the system are first written to a log file on the disk. Each log entry should contain enough information to redo or undo the modification. The log can be read on every restart to recover the previous state by replaying all the log entries. Using WAL results in a significantly reduced number of disk writes, because only the log file needs to be flushed to disk to guarantee that a transaction is committed, rather than every data file changed by the transaction.
Each node, in a distributed environment, maintains its own log. WAL is always sequentially appended, which simplifies the handling of the log.

<img align="center" width="50%" src="https://polybox.ethz.ch/index.php/s/un9OrcnS0Dw7GH2/download"><br>

What HBase does to ensure durability is whenever a node receives a write request, it immediately writes the data to a WAL stored on HDFS before writing data to a MemStore. This provides durability in the case of an unexpected shutdown, since all the data on the MemStore resides on volatile memory. On startup, the WAL will be replayed to repopulate the MemStores.

<img align="center" width="50%" src="https://polybox.ethz.ch/index.php/s/5Tvn3SMUs11bhqG/download"><br>

In summary, to ensure durability, LSM Trees use a **Write-Ahead Log**:

1. **Every write** (insert/update/delete) is **appended sequentially** to the WAL on disk.
2. The same write is then applied to the in-memory tree.

If the system crashes, the WAL can be **replayed** to reconstruct the in-memory tree. Because the WAL is **sequentially** written, it minimizes random I/O overhead.

- **Pros:** Ensures durability with relatively fast sequential writes.  
- **Cons:** Requires extra disk I/O for the log (although still quite efficient).
